In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as  F
import torch.nn as nn
from pathlib import Path
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from collections import OrderedDict
from tqdm import tqdm
import sys
import time
from sklearn.metrics import accuracy_score

In [2]:
DEVICE='cuda'
use_gpu=True

In [3]:
def reproducibilitySeed():
    """
    Ensure reproducibility of results; Seeds to 0
    """
    torch_init_seed = 0
    torch.manual_seed(torch_init_seed)
    numpy_init_seed = 0
    np.random.seed(numpy_init_seed)
    if use_gpu:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

reproducibilitySeed()

# # # *Loading Cifar10*

In [4]:


NUM_WORKERS = 2


class TensorImgSet(Dataset):
    """TensorDataset with support of transforms.
    """

    def __init__(self, tensors, transform=None):
        self.imgs = tensors[0]
        self.targets = tensors[1]
        self.tensors = tensors
        self.transform = transform
        self.len = len(self.imgs)

    def __getitem__(self, index):
        x = self.imgs[index]
        if self.transform:
            x = self.transform(x)
        y = self.targets[index]
        return x, y

    def __len__(self):
        return self.len



def get_cifar(num_classes=100, dataset_dir="./data", batch_size=128,
              use_cifar_10_1=False):

    if num_classes == 10:
        print("Loading CIFAR10...")
        dataset = torchvision.datasets.CIFAR10
        normalize = transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    else:
        print("Loading CIFAR100...")
        dataset = torchvision.datasets.CIFAR100
        normalize = transforms.Normalize(
            mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])

    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])

    trainset = dataset(root=dataset_dir, train=True,
                       download=True, transform=train_transform)

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    testset = dataset(root=dataset_dir, train=False,
                          download=True,
                          transform=test_transform)

    train_loader = torch.utils.data.DataLoader(trainset,
                                               batch_size=batch_size,
                                               num_workers=NUM_WORKERS,
                                               pin_memory=True, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset,
                                              batch_size=batch_size,
                                              num_workers=NUM_WORKERS,
                                              pin_memory=True, shuffle=False)
    return train_loader, test_loader

# # #*Creating Models*

In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)
        self.n_channels = [64, 128, 256, 512]

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, is_feat=False, use_relu=True):
        out = self.conv1(x)
        out = self.bn1(out)
        if use_relu:
            out = F.relu(out)
        feat1 = self.layer1(out)
        if use_relu:
            feat1 = F.relu(feat1)
        feat2 = self.layer2(feat1)
        if use_relu:
            feat2 = F.relu(feat2)
        feat3 = self.layer3(feat2)
        if use_relu:
            feat3 = F.relu(feat3)

        feat4 = self.layer4(feat3)
        feat4 = F.relu(feat4)
        pool = F.avg_pool2d(feat4, 4)
        pool = pool.view(pool.size(0), -1)
        out = self.linear(pool)

        if is_feat:
            return[feat1, feat2, feat3, feat4], pool, out

        return out

    def get_bn_before_relu(self):
        if isinstance(self.layer1[0], Bottleneck):
            bn1 = self.layer1[-1].bn3
            bn2 = self.layer2[-1].bn3
            bn3 = self.layer3[-1].bn3
            bn4 = self.layer4[-1].bn3
        elif isinstance(self.layer1[0], BasicBlock):
            bn1 = self.layer1[-1].bn2
            bn2 = self.layer2[-1].bn2
            bn3 = self.layer3[-1].bn2
            bn4 = self.layer4[-1].bn2
        else:
            print('ResNet unknown block error !!!')

        return [bn1, bn2, bn3, bn4]

    def get_channel_num(self):
        return self.n_channels

    def extract_feature(self, x, preReLU=False):

        x = self.conv1(x)
        x = self.bn1(x)

        feat1 = self.layer1(x)
        feat2 = self.layer2(feat1)
        feat3 = self.layer3(feat2)
        feat4 = self.layer4(feat3)

        x = F.relu(feat4)
        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)
        out = self.linear(x)

        if not preReLU:
            feat1 = F.relu(feat1)
            feat2 = F.relu(feat2)
            feat3 = F.relu(feat3)
            feat4 = F.relu(feat4)

        return [feat1, feat2, feat3, feat4], out


class ResNetSmall(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNetSmall, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(256 * block.expansion, num_classes)
        self.n_channels = [16, 32, 64]

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, is_feat=False, use_relu=True):
        out = self.conv1(x)
        out = self.bn1(out)
        if use_relu:
            out = F.relu(out)
        feat1 = self.layer1(out)
        if use_relu:
            feat1 = F.relu(feat1)
        feat2 = self.layer2(feat1)
        if use_relu:
            feat2 = F.relu(feat2)
        feat3 = self.layer3(feat2)

        # the last relu is always included
        feat3 = F.relu(feat3)
        pool = F.avg_pool2d(feat3, 4)
        pool = pool.view(pool.size(0), -1)
        out = self.linear(pool)

        if is_feat:
            return[feat1, feat2, feat3], pool, out

        return out

    def get_bn_before_relu(self):
        if isinstance(self.layer1[0], Bottleneck):
            bn1 = self.layer1[-1].bn3
            bn2 = self.layer2[-1].bn3
            bn3 = self.layer3[-1].bn3
        elif isinstance(self.layer1[0], BasicBlock):
            bn1 = self.layer1[-1].bn2
            bn2 = self.layer2[-1].bn2
            bn3 = self.layer3[-1].bn2
        else:
            print('ResNet unknown block error !!!')

        return [bn1, bn2, bn3]

    def get_channel_num(self):
        return self.n_channels

    def extract_feature(self, x, preReLU=False):

        x = self.conv1(x)
        x = self.bn1(x)

        feat1 = self.layer1(x)
        feat2 = self.layer2(feat1)
        feat3 = self.layer3(feat2)

        x = F.relu(feat3)
        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)
        out = self.linear(x)

        if not preReLU:
            feat1 = F.relu(feat1)
            feat2 = F.relu(feat2)
            feat3 = F.relu(feat3)

        return [feat1, feat2, feat3], out


def resnet8(**kwargs):
    return ResNetSmall(BasicBlock, [1, 1, 1], **kwargs)


def resnet14(**kwargs):
    return ResNetSmall(BasicBlock, [2, 2, 2], **kwargs)


def resnet20(**kwargs):
    return ResNetSmall(BasicBlock, [3, 3, 3], **kwargs)


def resnet26(**kwargs):
    return ResNetSmall(BasicBlock, [4, 4, 4], **kwargs)


def resnet32(**kwargs):
    return ResNetSmall(BasicBlock, [5, 5, 5], **kwargs)


def resnet44(**kwargs):
    return ResNetSmall(BasicBlock, [7, 7, 7], **kwargs)


def resnet56(**kwargs):
    return ResNetSmall(BasicBlock, [9, 9, 9], **kwargs)


def resnet10(**kwargs):
    return ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)


def resnet18(**kwargs):
    return ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)


def resnet34(**kwargs):
    return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)


def resnet50(**kwargs):
    return ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)


def resnet101(**kwargs):
    return ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)


def resnet152(**kwargs):
    return ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)


def test():
    net = resnet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [6]:
def load_checkpoint(model, checkpoint_path,device ):
    device = torch.device(device)
    model_ckp = torch.load(checkpoint_path, map_location=device)

    # handle both dataparallel and normal models
    model_tmp_dict = OrderedDict()
    for name, value in model_ckp["model_state_dict"].items():
        if name.startswith("module."):
            name = name[7:]
        model_tmp_dict[name] = value

    if isinstance(model, torch.nn.DataParallel):
        model.module.load_state_dict(model_tmp_dict)
    else:
        model.load_state_dict(model_tmp_dict)
    return model

In [7]:
teacher_model=resnet18()
#teacher_model=load_checkpoint(teacher_model,"../input/model-weights/resnet18_cifar10.pth",DEVICE)

In [8]:
teacher_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [9]:
#for param in teacher_model.parameters():
       # param.requires_grad=False

In [10]:
#for param in teacher_model.linear.parameters():
    #param.requires_grad = True

In [11]:
#import torchvision.models as models
#resnet18 = models.resnet18(pretrained=True)

In [12]:
#teacher_model=resnet18

In [13]:
#teacher_model.fc

In [14]:
#for param in teacher_model.parameters():
   # param.requires_grad = False

In [15]:
#teacher_model.fc= nn.Linear(512,10)

In [16]:
#teacher_model

In [17]:
learning_rate=0.1
num_epochs=200
optimizer_teacher =torch.optim.SGD(teacher_model.parameters(), lr=1e-1, momentum=0.9, weight_decay=0.0005, nesterov=True)
scheduler_teacher = torch.optim.lr_scheduler.MultiStepLR(optimizer_teacher, milestones=[66,132], gamma=0.1,verbose=True)

Adjusting learning rate of group 0 to 1.0000e-01.


In [18]:
def freeze_teacher(t_net):
    # freeze the layers of the teacher
    for param in t_net.parameters():
        param.requires_grad = False
    # set the teacher net into evaluation mode
    t_net.eval()
    return t_net

In [19]:
def init_progress_bar(train_loader):
    batch_size = train_loader.batch_size
    bar_format = "{desc}{percentage:3.0f}%"
    # bar_format += "|{bar}|"
    bar_format += " {n_fmt}/{total_fmt} [{elapsed} < {remaining}]"
    bar_format += "{postfix}"
    # if stderr has no tty disable the progress bar
    disable = not sys.stderr.isatty()
    t = tqdm(total=len(train_loader) * batch_size,
             bar_format=bar_format)
    if disable:
        # a trick to allow execution in environments where stderr is redirected
        t._time = lambda: 0.0
    return t

In [20]:
print(init_progress_bar)

<function init_progress_bar at 0x7f5c58157d40>


In [21]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

In [22]:
def train_fn(train_loader, model, criterion, optimizer, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()


    model.train()
    start = end = time.time()
    for step, (images, labels) in tqdm(enumerate(train_loader), total = len(train_loader)):
        data_time.update(time.time() - end)
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)
        batch_size = labels.size(0)

        y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch_time.update(time.time() - end)
        end = time.time()

    return losses.avg

def valid_fn(test_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()


    model.eval()
    start = end = time.time()
    preds = []
    valid_labels = []
    for step, (images, labels) in tqdm(enumerate(test_loader), total = len(test_loader)):
        data_time.update(time.time() - end)
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)
        batch_size = labels.size(0)

        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)

        preds.append(y_preds.softmax(1).to('cpu').numpy())
        valid_labels.append(labels.to('cpu').numpy())
        
        batch_time.update(time.time() - end)
        end = time.time()

    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    
    score = get_score(valid_labels, predictions.argmax(1))
    
    return losses.avg, score

In [23]:
def calculate_loss(model, data, target):
        # Standard Learning Loss ( Classification Loss)
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer_teacher.step()
        return output, loss

In [24]:
 def save(model, epoch, name):
        torch.save({"model_state_dict": model.state_dict(), }, name)

In [25]:
train_loader,test_loader=get_cifar(num_classes=10, dataset_dir="./data", batch_size=128,
              use_cifar_10_1=False)

Loading CIFAR10...


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [26]:
criterion = nn.CrossEntropyLoss()

In [27]:
best_score = 0

for epoch in range(200):
    start_time = time.time()
    teacher_model.to(DEVICE)
    avg_loss = train_fn(train_loader, teacher_model, criterion, optimizer_teacher, None, DEVICE)
    avg_val_loss, score = valid_fn(test_loader, teacher_model, criterion, DEVICE)

    scheduler_teacher.step()
    
    print(f"Epoch {epoch+1}")
    print(f"Accuracy: {score} | Train loss: {avg_loss} | Valid loss: {avg_val_loss}")
    
    if score > best_score:
        print("YES")
        best_score = score
        torch.save(teacher_model.state_dict(), f"best_model.pt")
        
    elapsed = time.time() - start_time

100%|██████████| 79/79 [00:02<00:00, 27.97it/s]


Adjusting learning rate of group 0 to 1.0000e-01.
Epoch 1
Accuracy: 0.4698 | Train loss: 1.797891656227112 | Valid loss: 1.4240377485275268
YES


100%|██████████| 79/79 [00:03<00:00, 24.77it/s]


Adjusting learning rate of group 0 to 1.0000e-01.
Epoch 2
Accuracy: 0.5805 | Train loss: 1.2592047958755492 | Valid loss: 1.1614779123306274
YES


100%|██████████| 79/79 [00:02<00:00, 28.36it/s]


...

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 198
Accuracy: 0.9514 | Train loss: 0.0015371468069031835 | Valid loss: 0.1893610878944397



100%|██████████| 79/79 [00:03<00:00, 26.16it/s]


Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 199
Accuracy: 0.9521 | Train loss: 0.0016757110823132097 | Valid loss: 0.1892338653087616


100%|██████████| 79/79 [00:02<00:00, 29.11it/s]

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 200
Accuracy: 0.9522 | Train loss: 0.001640079644396901 | Valid loss: 0.19031467709541322


In [28]:
best_score

0.9527